In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import os
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet ###REMOVE
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [57]:
# Defining the seasons and models used
seasons = ['2023-24','2022-23',
           '2021-22','2020-21','2019-20','2018-19','2017-18','2016-17','2015-16','2014-15','2013-14',
          '2012-13','2011-12','2010-11','2009-10','2008-09','2007-08','2006-07'
          ]
seasons.reverse()
           
models = ['SVM', 'XGBoost','Random Forest','AdaBoost','Gradient Boosting',]#'LGBM']

path = current_path = os.getcwd()
# Path to local folder
path_data = '/Users/von/Desktop/Work/Projects/NBA_MIP_Predictor/NBA-MIP-Model'

sep = r'/'

In [58]:
def get_previous_season(current_season):
    years = current_season.split('-')
    
    first_year = str(int(years[0]) - 1)
    second_year = '{:02d}'.format(int(years[1]) - 1)
    return first_year + '-' + second_year

In [59]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 80)

def get_data(seasons):
    
    # Opening the data
    per_game = pd.DataFrame()
    totals = pd.DataFrame()
    difference = pd.DataFrame()
    advanced_stats = pd.DataFrame()
    standings = pd.DataFrame()
    difference_pergame = pd.DataFrame()
    difference_advanced = pd.DataFrame()

    for season in seasons:
        if season == '2004-05':
            break

        updated_path = '/Updated Per Game Seasons'
        updated = "-Updated"
        pergame = pd.read_csv(path_data+sep+'Data'+updated_path+sep+season+'-PerGame'+updated+'.csv',
                               sep=',', decimal='.')

        total = pd.read_csv(path_data+sep+'Data'+sep+season+' Total.csv',
                            sep=',', decimal='.')
        
        advanced = pd.read_csv(path_data+sep+'Data'+updated_path+sep+season+'-Advanced'+updated+'.csv',
                               sep=',', decimal='.')
        
        pergame_previous = pd.read_csv(path_data+sep+'Data'+updated_path+sep+get_previous_season(season)+'-PerGame'+updated+'.csv',
                               sep=',', decimal='.')
                               
        advanced_previous = pd.read_csv(path_data+sep+'Data'+updated_path+sep+get_previous_season(season)+'-Advanced'+updated+'.csv',
                               sep=',', decimal='.')
        
        standing = pd.read_csv(path_data+sep+'Data'+updated_path+sep+season+'-Standings'+updated+'.csv',
                               sep=',', decimal='.')
        
        
        # Getting dataframe for difference of stat averages from previous year
        cols_to_diff_pergame = ['G','GS','MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA','TRB','AST','BLK','TOV','PTS','DRB']
        cols_to_diff_advanced = ['PER','3PAr','ORB%','AST%','STL%','BLK%','USG%','OWS','DWS','WS','WS/48','OBPM','DBPM','BPM','VORP']


        ### Was better not dropping these
        '''
        drop_list = ['3PAr', 'BLK%', 'STL%', 'ORB%', 'OBPM', 'DBPM']
        for stat in drop_list:
            cols_to_diff_advanced.remove(stat)
        '''

        # First, make sure all Players' names are the same in every data file used
        # Using str.replace() to remove all text from *\ or \ onwards
        pergame['Player'] = pergame['Player'].str.replace(r'[\*\\].*', '', regex=True)
        pergame_previous['Player'] = pergame_previous['Player'].str.replace(r'[\*\\].*', '', regex=True)
        advanced['Player'] = advanced['Player'].str.replace(r'[\*\\].*', '', regex=True)
        advanced_previous['Player'] = advanced_previous['Player'].str.replace(r'[\*\\].*', '', regex=True)

        # Remove duplicate players, keeping only the first occurrence
        pergame= pergame.drop_duplicates(subset='Player', keep='first')
        pergame_previous= pergame_previous.drop_duplicates(subset='Player', keep='first')
        advanced= advanced.drop_duplicates(subset='Player', keep='first')
        advanced_previous= advanced_previous.drop_duplicates(subset='Player', keep='first')

        # Merge current and past season to get stats differences
        difference_pergame_season = pd.merge(pergame, pergame_previous, on='Player', suffixes=('_Current', '_Previous'), how='inner')
        difference_advanced_season = pd.merge(advanced, advanced_previous, on='Player', suffixes=('_Current', '_Previous'), how='inner')
        
        #difference_pergame_season = difference_pergame_season.drop_duplicates(subset='Player', keep='first')

        diff_pergame_list = [
            'Player', 'G_diff','GS_diff', 'MP_diff', 'FG_diff', 'FGA_diff', 
            '3P_diff', '3PA_diff', '2P_diff', '2PA_diff', 'FT_diff', 'FTA_diff', 
            'DRB_diff', 'TRB_diff', 'AST_diff', 
            'BLK_diff', 'TOV_diff', 'PTS_diff', 
            'PTS_diff_BS'

        ]
        short_diff_pergame_list = [
            'Player','TRB_diff', 'AST_diff', 'PTS_diff',
        ]
        diff_advanced_list = [
            'Player', 'PER_diff','AST%_diff','USG%_diff','OWS_diff',
            'STL%_diff','BLK%_diff','3PAr_diff','ORB%_diff','OBPM_diff','DBPM_diff',
            'DWS_diff','WS_diff','WS/48_diff','BPM_diff','VORP_diff', 
        ]
        #short_advanced_list

        diff_pergame_percent_list = [
            'Player', 'G_PCT_diff','GS_PCT_diff', 'MP_PCT_diff', 'FG_PCT_diff', 'FGA_PCT_diff', 
            '3P_PCT_diff', '3PA_PCT_diff', '2P_PCT_diff', '2PA_PCT_diff', 'FT_PCT_diff', 'FTA_PCT_diff', 
            'DRB_PCT_diff', 'TRB_PCT_diff', 'AST_PCT_diff', 
            'BLK_PCT_diff', 'TOV_PCT_diff', 'PTS_PCT_diff', 'TRB_diff', 'AST_diff', 'PTS_diff',
        ]
        diff_advanced_percent_list = [
            'Player', 'PER_PCT_diff','AST%_PCT_diff','USG%_PCT_diff','OWS_PCT_diff',
            'STL%_PCT_diff','BLK%_PCT_diff','DBPM_PCT_diff','OBPM_PCT_diff','3PAr_PCT_diff','ORB%_PCT_diff',
            'DWS_PCT_diff','WS_PCT_diff','WS/48_PCT_diff','BPM_PCT_diff','VORP_PCT_diff', 
        ]
        
            


        joined_pergames = diff_pergame_list + diff_pergame_percent_list
        joined_diffs_pergame_list = set(joined_pergames)
        joined_advanced = diff_advanced_list + diff_advanced_percent_list
        joined_diffs_advanced_list = set(joined_advanced)
        # Perform subtraction for specific columns
        get_standard_diffs = True
        get_pct_diffs = False
        #
        #
        #print(difference_pergame_season.columns)
        if get_standard_diffs and not get_pct_diffs:
            for col in cols_to_diff_pergame:
                difference_pergame_season[col + '_diff'] = difference_pergame_season[col + '_Current'] - difference_pergame_season[col + '_Previous']

            # Try to get both ways 
            try:
                difference_pergame_season['PTS_diff_BS'] = difference_pergame_season['PTS_Current'] - difference_pergame_season['BS_PTS_Current']
            except:
                difference_pergame_season['PTS_diff_BS'] = difference_pergame_season['PTS_Current'] - difference_pergame_season['BS_PTS']
            difference_pergame_season = difference_pergame_season[diff_pergame_list]
           # print(difference_pergame_season.columns)
            
            
            for col in cols_to_diff_advanced:
                difference_advanced_season[col + '_diff'] = difference_advanced_season[col + '_Current'] - difference_advanced_season[col + '_Previous']
            difference_advanced_season = difference_advanced_season[diff_advanced_list]



        # Perform % difference for specific columns
        #
        #
        elif get_pct_diffs and not get_standard_diffs:
            for col in cols_to_diff_pergame:
                difference_pergame_season[col + '_PCT_diff'] = (difference_pergame_season[col + '_Current'] - difference_pergame_season[col + '_Previous']) / difference_pergame_season[col + '_Previous']
                
                #Account for possible inf/-inf values
                difference_pergame_season[col + '_PCT_diff'].replace([np.inf, -np.inf], 10.0, inplace=True) 
            
            if not get_standard_diffs:
                ### At minimum need to add PTS, TRB, and AST normal difference
                difference_pergame_season['PTS_diff'] = difference_pergame_season['PTS_Current'] - difference_pergame_season['PTS_Previous']
                difference_pergame_season['AST_diff'] = difference_pergame_season['AST_Current'] - difference_pergame_season['AST_Previous']
                difference_pergame_season['TRB_diff'] = difference_pergame_season['TRB_Current'] - difference_pergame_season['TRB_Previous']
            
            if not get_standard_diffs:
                difference_pergame_season = difference_pergame_season[diff_pergame_percent_list]
            else:
                difference_pergame_season = difference_pergame_season[joined_diffs_pergame_list]
            
            for col in cols_to_diff_advanced:
                difference_advanced_season[col + '_PCT_diff'] = (difference_advanced_season[col + '_Current'] - difference_advanced_season[col + '_Previous']) / difference_advanced_season[col + '_Previous']
                #Account for possible inf/-inf values
                difference_advanced_season[col + '_PCT_diff'].replace([np.inf, -np.inf], 10.0, inplace=True) 

            if not get_standard_diffs:
                difference_advanced_season = difference_advanced_season[diff_advanced_percent_list]
            else:
                difference_advanced_season = difference_advanced_season[joined_diffs_advanced_list]
                
        else:
            for col in cols_to_diff_pergame:
                difference_pergame_season[col + '_diff'] = difference_pergame_season[col + '_Current'] - difference_pergame_season[col + '_Previous']
            for col in cols_to_diff_advanced:
                difference_advanced_season[col + '_diff'] = difference_advanced_season[col + '_Current'] - difference_advanced_season[col + '_Previous']

            for col in cols_to_diff_pergame:
                difference_pergame_season[col + '_PCT_diff'] = (difference_pergame_season[col + '_Current'] - difference_pergame_season[col + '_Previous']) / difference_pergame_season[col + '_Previous']
                
                #Account for possible inf/-inf values
                difference_pergame_season[col + '_PCT_diff'].replace([np.inf, -np.inf], 10.0, inplace=True) 

            difference_pergame_season = difference_pergame_season[joined_diffs_pergame_list]
            
            for col in cols_to_diff_advanced:
                difference_advanced_season[col + '_PCT_diff'] = (difference_advanced_season[col + '_Current'] - difference_advanced_season[col + '_Previous']) / difference_advanced_season[col + '_Previous']
                
                #Account for possible inf/-inf values
                difference_advanced_season[col + '_PCT_diff'].replace([np.inf, -np.inf], 10.0, inplace=True) 

            difference_advanced_season = difference_advanced_season[joined_diffs_advanced_list]

        
        #difference_advanced_season['Player-additional'] = difference_advanced_season['Player-additional_Current']

        
        pergame['Season'] = season
        total['Season'] = season
        advanced['Season'] = season
        standing['Season'] = season
        difference_advanced_season['Season'] = season
        difference_pergame_season['Season'] = season


        
        per_game = pd.concat([per_game,pergame], ignore_index=True)
        totals = pd.concat([totals,total], ignore_index=True)
        advanced_stats = pd.concat([advanced_stats,advanced], ignore_index=True)
        standings = pd.concat([standings,standing], ignore_index=True)
        difference_pergame = pd.concat([difference_pergame, difference_pergame_season], ignore_index=True)
        difference_advanced = pd.concat([difference_advanced, difference_advanced_season], ignore_index=True)
        

        
        #difference = difference.merge(difference_advanced, on=['Player','Season','Tm'], how='left', validate='1:1')
    #print(difference_pergame)
   # print(difference_pergame.duplicated(subset=["Player"]).sum())
   #print(totals)
    #print(pergame)
    #print
    

    #Check occurances of a player
    #player_counts = difference_pergame['Player'].value_counts()
    #count = player_counts.get("DeMar DeRozan", 0)
    #print(count)


    # Need to merge advanced stats and pergame stats differences into one dataframe
    difference = pd.merge(difference_pergame, difference_advanced, on=['Player', 'Season'], how='outer', suffixes=('_pergame', '_advanced'))

    return per_game, totals, advanced_stats, standings, difference


In [60]:
def treat_data(per_game, totals, advanced, difference, standings, seasons):
    
    # Removing duplicate/empty columns
    per_game = per_game.drop(['Rk','Pos'], axis=1)
    totals = totals.drop(['Rk','Pos','Age','G','GS'], axis=1)
    advanced = advanced.drop(['Rk','Pos','Age','G','MP','Unnamed: 24','Unnamed: 19', 'Player-additional'], axis=1)

    # Test 2
    # Remove more variables
    per_game = per_game.drop(['GS', 'PF', 'STL', 'eFG%', 'FTA', 'G_prev_total', 'Best Season', 'Previous Season',], axis=1)
    #Need to drop these also
    per_game = per_game.drop(['PS_PTS','PS_TRB','PS_AST','PS_STL','PS_BLK','PS_MP','PS_G','PS_FGA','PS_FG','PS_TOV','PS_3P','PS_3PA',
                            'PS_2P','PS_3P%','PS_2PA','PS_2P%','PS_FT%','BS_PTS','BS_TRB','BS_AST','BS_STL','BS_BLK',
                              'BS_MP','BS_G','BS_FGA','BS_FG','BS_TOV','BS_3P','BS_3PA','BS_2P','BS_3P%','BS_2PA','BS_2P%','BS_FT%'], axis=1)
    totals = totals.drop(['PF'], axis=1)
    
    advanced = advanced.drop(['TRB%', 'DBPM', 'OBPM', 'FTr', 'STL%', 'DRB%', 'BLK%', 'TS%', 'DWS', 'TOV%', 'ORB%', '3PAr', 'Best Season', 
                              'Previous Season'], axis=1)
    #Need to drop these also
    advanced = advanced.drop(['PS_PER','PS_TRB%','PS_AST%','PS_BPM','PS_VORP','PS_USG%','PS_WS','PS_WS/48','BS_PER',
                              'BS_TRB%','BS_AST%','BS_BPM','BS_VORP','BS_USG%','BS_WS','BS_WS/48'], axis=1)

    cols = ['Player','Season','Pos','Age','Tm','G','GS', 'G_prev', 'G_prev_total', 'Seasons Played']

    previous_winners = []
    
    # Identifying the variables
    for col in per_game.columns:
        if col not in cols:
            new_col = col+'_PERGAME'
            per_game = per_game.rename(columns={col:new_col})
    for col in totals.columns:
        if col not in cols:
            new_col = col+'_TOTAL'
            totals = totals.rename(columns={col:new_col})
    for col in advanced.columns:
        if col not in cols:
            new_col = col+'_ADVANCED'
            advanced = advanced.rename(columns={col:new_col})
    
    # Merging the bases
    data = per_game.merge(advanced, on=['Player','Season','Tm'], how='left', validate='1:1')
    
    # Not using Totals anymore
    #data = data.merge(totals, on=['Player','Season','Tm'], how='left', validate='1:1')
    
    #Not using differences for now
    data = data.merge(difference, on=['Player','Season',], how='left', validate='1:1')
    print(data.columns)
    
    dict_teams = {'Utah Jazz':'UTA','Phoenix Suns':'PHO',
                 'Philadelphia 76ers':'PHI','Brooklyn Nets':'BRK',
                 'Denver Nuggets':'DEN','Los Angeles Clippers':'LAC',
                 'Milwaukee Bucks':'MIL','Dallas Mavericks':'DAL',
                 'Los Angeles Lakers':'LAL','Portland Trail Blazers':'POR',
                 'Atlanta Hawks':'ATL','New York Knicks':'NYK',
                 'Miami Heat':'MIA','Golden State Warriors':'GSW',
                 'Memphis Grizzlies':'MEM','Boston Celtics':'BOS',
                 'Washington Wizards':'WAS','Indiana Pacers':'IND',
                 'Charlotte Hornets':'CHO','Charlotte Bobcats':'CHA',
                 'San Antonio Spurs':'SAS','Chicago Bulls':'CHI',
                 'New Orleans Pelicans':'NOP','Sacramento Kings':'SAC',
                 'Toronto Raptors':'TOR','Minnesota Timberwolves':'MIN',
                 'Cleveland Cavaliers':'CLE','Oklahoma City Thunder':'OKC',
                 'Orlando Magic':'ORL','Detroit Pistons':'DET',
                 'Houston Rockets':'HOU','New Jersey Nets':'NJN',
                 'New Orleans Hornets':'NOH','Seattle SuperSonics':'SEA'}
    
    teams = pd.DataFrame.from_dict(dict_teams, orient='index').reset_index()
    teams = teams.rename(columns={'index':'Team',0:'Tm'})
    standings = standings.merge(teams, on='Team', how='left', validate='m:1')
    wins = (standings['Record'].str.split('-',expand=True)[0]).astype(int)
    games = ((standings['Record'].str.split('-',expand=True)[0]).astype(int)+(standings['Record'].str.split('-',expand=True)[1]).astype(int))
    standings['PCT'] = wins/games
    
    data = data.merge(standings, on=['Tm','Season'], how='left', validate='m:1')
    
    data['Player'] = data['Player'].str.replace('*','')
    
    ###mvps = pd.read_csv(path_data+sep+'Dados'+sep+'MVPs.csv',sep=',', decimal='.') ###Change to MIPS

    mips = pd.read_csv(path_data+sep+'Data'+sep+'MIPs.csv',sep=',', decimal='.')


    ###data = data.merge(mvps, on=['Player','Season'], how='left', validate='m:1').fillna(0)  
    data = data.merge(mips, on=['Player','Season'], how='left', validate='m:1').fillna(0) 
    data = data.drop(['Won already'], axis=1)

    data['Player'] = data['Player'].str.split('\\', expand=True)[0]
    #print(data)
    
    # Removing duplicate lines from traded players, removing players from seasons that have already won the award
    dataf = pd.DataFrame()
    
    for season in seasons:
        data_season = data[data['Season']==season]
        data_season = data_season.drop_duplicates(subset=['Player'], keep='first')

        # Remove players who already won
        data_season = data_season[~data_season['Player'].isin(previous_winners)]

        result = mips[(mips['MIP Rank'] == '1') & (mips['Season'] == season)]
        #print(result)
        #print(mips[(mips['MIP Rank'] == '1') & (mips['Season'] == season)])
        
        # Add winning player to winners list
        winner_name = result['Player'].tolist()[0]
        if winner_name not in previous_winners:
            previous_winners.append(winner_name)
            #print(winner_name)
        
        
        #print(season)
        dataf = pd.concat([dataf,data_season], ignore_index=True)

    print(dataf.columns)

    # Filtering the data (MVP)
    '''
    dataf = dataf[((dataf['G']>48)&(dataf['PTS_PERGAME']>13.5)&(dataf['MP_PERGAME']>30)
                 &(dataf['Seed']<=16)&(dataf['AST_PERGAME']>1)&(dataf['TRB_PERGAME']>3)
                 &(dataf['Tm']!='TOT')&(dataf['FG%_PERGAME']>0.37)&(dataf['FGA_PERGAME']>10)
                 &(dataf['PER_ADVANCED']>18)) | (dataf['MVP Votes Share']>0)].reset_index(drop=True)
    '''
    # Filtering the data (MIP)
    #print(dataf)
    dataf = dataf[((dataf['G']>50)&(dataf['Age'] <= 30)&(dataf['PTS_PERGAME']>=9.7)&(dataf['G_prev']>45)
                   &(dataf['FGA_PERGAME']>=7.8)&(dataf['MP_PERGAME']>=22.9)&(dataf['PER_ADVANCED']>=14.2)
                   &(dataf['TRB_PERGAME']>=2.6)&(dataf['AST_PERGAME']>=0.7)&(dataf['FG%_PERGAME']>0.41)
                   &(dataf['PTS_diff']>=1.3)&(dataf['TRB_diff']>=-1.0)&(dataf['PER_ADVANCED']<=27.7) 
                   &(dataf['AST_diff']>=-1.5)&(dataf['PTS_PERGAME']<=27.4)) | ((dataf['MIP Share']>=0.1)&(dataf['G_prev']>35))].reset_index(drop=True)
    
    # Base for the criteria:
    # Karl Malone was MVP in 98-99 with 49 games
    # Wes Unseld was MVP at 68-69 with 13.8 PPG and with 10.9 FGA
    # Steve Nash was MVP at 04-05 with 3.3 REB
    # Moses Malone was MVP at 82-83 with 1.3 AST
    # Bob Cousy was MVP at 56-57 with 37.8% FG
    # Giannis Antetokounmpo was MVP in 19-20 with 30.4 min
    # Kareem Abdul-Jabbar was the only MVP not to make the playoffs in 1976 (40-42)
    # Dave Cowens was MVP at 72-73 with a PER of 18.1
    # Never has an MVP been traded in the middle of the season that he won the award


    # Base for the MIP criteria
    # Darrell Armstrong was MIP in 98-99 with 50 games, oldest age 30
    # Isaac Austin was MIP in 96-97 with 9.7 PPG, 7.8 FGA, 22.9 MPG, 14.2 PER
    # Aaron Brooks was MIP in 2009-10 with 2.6 RPG
    # Gheorghe Muresan was MIP in 95-96 with 0.7 APG
    # Paul George was MIP in 2012-13 with 41.9% FG
    # 
    
    # Since 2001-02
    # PPG difference >= 4.5
    # PPG diff of top 3 candidates >= 1.3
    # RPG diff top 3 >=  -1.0
    # APG diff top3 >= -1.5
    # Season before >= 41 games played
    # 

    

        
    dataf = dataf.drop(['Tm','Team',
                        'Record'
    ], 
                        axis=1)
    dataf.to_csv("FinalDataFrame.csv", index=False)
    
    return dataf

In [61]:
pd.options.display.max_seq_items = 203

per_game, totals, advanced, standings, difference = get_data(seasons)



#print("Breaks")
#per_game

In [62]:
data = treat_data(per_game, totals, advanced, difference, standings, seasons)

Index(['Player', 'Age', 'Tm', 'G', 'MP_PERGAME', 'FG_PERGAME', 'FGA_PERGAME',
       'FG%_PERGAME', '3P_PERGAME', '3PA_PERGAME', '3P%_PERGAME', '2P_PERGAME',
       '2PA_PERGAME', '2P%_PERGAME', 'FT_PERGAME', 'FT%_PERGAME',
       'ORB_PERGAME', 'DRB_PERGAME', 'TRB_PERGAME', 'AST_PERGAME',
       'BLK_PERGAME', 'TOV_PERGAME', 'PTS_PERGAME', 'G_prev', 'Seasons Played',
       'Season', 'PER_ADVANCED', 'AST%_ADVANCED', 'USG%_ADVANCED',
       'OWS_ADVANCED', 'WS_ADVANCED', 'WS/48_ADVANCED', 'BPM_ADVANCED',
       'VORP_ADVANCED', 'G_diff', 'GS_diff', 'MP_diff', 'FG_diff', 'FGA_diff',
       '3P_diff', '3PA_diff', '2P_diff', '2PA_diff', 'FT_diff', 'FTA_diff',
       'DRB_diff', 'TRB_diff', 'AST_diff', 'BLK_diff', 'TOV_diff', 'PTS_diff',
       'PTS_diff_BS', 'PER_diff', 'AST%_diff', 'USG%_diff', 'OWS_diff',
       'STL%_diff', 'BLK%_diff', '3PAr_diff', 'ORB%_diff', 'OBPM_diff',
       'DBPM_diff', 'DWS_diff', 'WS_diff', 'WS/48_diff', 'BPM_diff',
       'VORP_diff'],
      dtype='object')


In [93]:
pd.options.display.max_rows = 532
#data
data[data["MIP Rank"].isin(["1", "2", "2T"])][["MIP Rank","Player","Season", "Seed", "PTS_diff", "PTS_diff_BS", 'USG%_ADVANCED','USG%_diff']]

,MIP Rank,Player,Season,Seed,PTS_diff,PTS_diff_BS,USG%_ADVANCED,USG%_diff
13,1,Monta Ellis,2006-07,12.0,9.7,9.7,22.4,2.2
23,2,Kevin Martin,2006-07,23.0,9.4,9.4,23.2,5.8
41,2,Rudy Gay,2007-08,27.0,9.3,9.3,25.0,3.7
60,1,Hedo Türkoğlu,2007-08,9.0,6.2,4.6,24.8,3.1
76,1,Danny Granger,2008-09,18.0,6.2,6.2,29.6,6.4
78,2,Devin Harris,2008-09,21.0,6.5,6.5,28.4,5.8
100,1,Aaron Brooks,2009-10,16.0,8.4,8.4,25.7,2.8
102,2T,Kevin Durant,2009-10,10.0,4.8,4.8,32.0,3.7
104,2T,Marc Gasol,2009-10,18.0,2.7,2.7,16.9,-1.4
106,2T,George Hill,2009-10,12.0,6.7,6.7,19.0,-0.5


In [64]:
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 4000
data.columns

Index(['Player', 'Age', 'G', 'MP_PERGAME', 'FG_PERGAME', 'FGA_PERGAME',
       'FG%_PERGAME', '3P_PERGAME', '3PA_PERGAME', '3P%_PERGAME', '2P_PERGAME',
       '2PA_PERGAME', '2P%_PERGAME', 'FT_PERGAME', 'FT%_PERGAME',
       'ORB_PERGAME', 'DRB_PERGAME', 'TRB_PERGAME', 'AST_PERGAME',
       'BLK_PERGAME', 'TOV_PERGAME', 'PTS_PERGAME', 'G_prev', 'Seasons Played',
       'Season', 'PER_ADVANCED', 'AST%_ADVANCED', 'USG%_ADVANCED',
       'OWS_ADVANCED', 'WS_ADVANCED', 'WS/48_ADVANCED', 'BPM_ADVANCED',
       'VORP_ADVANCED', 'G_diff', 'GS_diff', 'MP_diff', 'FG_diff', 'FGA_diff',
       '3P_diff', '3PA_diff', '2P_diff', '2PA_diff', 'FT_diff', 'FTA_diff',
       'DRB_diff', 'TRB_diff', 'AST_diff', 'BLK_diff', 'TOV_diff', 'PTS_diff',
       'PTS_diff_BS', 'PER_diff', 'AST%_diff', 'USG%_diff', 'OWS_diff',
       'STL%_diff', 'BLK%_diff', '3PAr_diff', 'ORB%_diff', 'OBPM_diff',
       'DBPM_diff', 'DWS_diff', 'WS_diff', 'WS/48_diff', 'BPM_diff',
       'VORP_diff', 'Seed', 'Previous Seed', 'See

In [65]:
data['Season'].value_counts() # Number of players in the data per season\

2013-14    42
2016-17    35
2006-07    34
2010-11    33
2017-18    32
2008-09    31
2018-19    31
2007-08    31
2012-13    29
2020-21    28
2019-20    28
2015-16    28
2014-15    26
2021-22    26
2009-10    25
2022-23    25
2023-24    24
2011-12    23
Name: Season, dtype: int64

In [66]:
# Function for metrics
def func_metrics(y_test, y_pred, metrics, model, season):
    rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)),3) # RMSE
    r2 = round(r2_score(y_test, y_pred),3) # R²
    
    dict_met = {'Model': [model],
                'Season': [season],
                'RMSE': [rmse],
                'R²': [r2]}
    
    metric = pd.DataFrame(data=dict_met)
    metrics = pd.concat([metrics,metric])
    
    return metrics

In [67]:
def func_models(data, seasons, models, n_seasons_to_test):
    final_results = pd.DataFrame()
    metrics = pd.DataFrame()
    best_params = []
    i = 1

    for season in seasons:

        # Separating training and testing bases
        season_test = season

        data_train = data[data['Season']!=season_test]
        data_test = data[data['Season']==season_test]


        regular_diffs = ['PTS_diff', 'AST_diff', 'TRB_diff']
        final_drops = []

        # Test 2
        X_train = data_train.drop(['MIP Share','MIP Rank','Player','Season',] + final_drops, axis=1)#'G_prev','FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME'], axis=1)
        y_train = data_train['MIP Share']

        X_test = data_test.drop(['MIP Share','MIP Rank','Player','Season',]+ final_drops, axis=1) #'G_prev','FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME'], axis=1)
        y_test = data_test['MIP Share']

        initial_results = data_test[['Player','Season','MIP Share','MIP Rank']]
        results = initial_results.copy()
        print(len(X_train.columns))

        scaler = StandardScaler()

        scaled_X_train = scaler.fit_transform(X_train)
        scaled_X_test = scaler.transform(X_test)

        for model in models:
            # Creating instance for each model
            if model=='SVM':
                param_grid = {'C': [0.001,0.01,0.1,0.5,1,2,5,10],
                             'kernel': ['linear','rbf','poly'],
                             'gamma': ['scale','auto'],
                             'degree': [2,3,4],
                             'epsilon': [0.1,0.5,1]}
                svr_model = SVR()
                grid = GridSearchCV(svr_model, param_grid)
                grid.fit(scaled_X_train, y_train)
                final_model = SVR(**grid.best_params_)
                best_params.append(grid.best_params_)

            elif model=='Elastic Net':
                param_grid = {'alpha':[0.01,0.1,1,5,10,50,100],
                              'l1_ratio':[0.01,0.1,0.5,0.7,0.95,0.99,1]}
                elastic_net_model = ElasticNet()
                grid = GridSearchCV(elastic_net_model, param_grid)
                grid.fit(scaled_X_train, y_train)
                grid.best_params_
                final_model = ElasticNet(**grid.best_params_)
                best_params.append(grid.best_params_)
            elif model=='XGBoost':
                param_grid = {'n_estimators': [100, 200],
                              'learning_rate': [0.01, 0.1],
                              'max_depth': [3, 5, 7],
                              'subsample': [0.7, 1.0]}
                xg_boost = XGBRegressor()
                grid = GridSearchCV(xg_boost, param_grid)
                grid.fit(scaled_X_train, y_train)
                grid.best_params_
                final_model = XGBRegressor(**grid.best_params_)
                best_params.append(grid.best_params_)
                
            elif model=='Random Forest':
                param_grid = {'n_estimators': [15,25,50,64,100,150,200],
                             'max_features': [2,3,4,5],
                             'bootstrap': [True,False],
                             'oob_score': [True]}
                rfc = RandomForestRegressor()
                grid = GridSearchCV(rfc, param_grid)
                grid.fit(scaled_X_train, y_train)
                final_model = RandomForestRegressor(**grid.best_params_)
                best_params.append(grid.best_params_)
                
            elif model=='AdaBoost':
                param_grid = {'n_estimators': [5,10,20,30,40,50,100],
                             'learning_rate': [0.01,0.05,0.1,0.2,0.5]}
                ada_model = AdaBoostRegressor()
                grid = GridSearchCV(ada_model, param_grid)
                grid.fit(scaled_X_train, y_train)
                final_model = AdaBoostRegressor(**grid.best_params_)
                best_params.append(grid.best_params_)
                
            elif model=='Gradient Boosting':
                param_grid = {'n_estimators': [10,20,30,40,50],
                             'learning_rate': [0.01,0.05,0.1,0.2,0.5],
                             'max_depth': [3,4,5]}
                gb_model = GradientBoostingRegressor()
                grid = GridSearchCV(gb_model, param_grid)
                grid.fit(scaled_X_train, y_train)
                final_model = GradientBoostingRegressor(**grid.best_params_)
                best_params.append(grid.best_params_)
                
            elif model=='LGBM':
                param_grid = {'learning_rate':[0.01,0.1,0.2,0.3],
                              'num_leaves':[5,10,20,30],
                              'min_data_in_leaf':[10,25,50],
                              'min_child_samples': [10, 25, 50],
                              'max_depth':[2,3,4],
                              'feature_fraction':[0.6,0.7,0.8,0.9],
                              'min_gain_to_split':[0,0.01,0.1,0.2],
                              'verbose':[-1]}
                lgbm_model = LGBMRegressor()
                grid = GridSearchCV(lgbm_model, param_grid)
                grid.fit(scaled_X_train, y_train)
                final_model = LGBMRegressor(**grid.best_params_)
                best_params.append(grid.best_params_)
                
            final_model.fit(scaled_X_train, y_train)
            
            print("FM:")
            print(final_model)
            print("SEP: ")
            print(sep)
            print("MODEL: ")
            print(model)
            pickle.dump(final_model,open(path_data+sep+'Models'+sep+model+'.dat','wb'))#,'wb')
            
            y_pred = final_model.predict(scaled_X_test)
            metrics = func_metrics(y_test, y_pred, metrics, model, season)

            temp_df = initial_results.copy()
            temp_df['Predicted MIP Share '+model] = pd.Series(y_pred).values

            results_sorted = temp_df.sort_values(by='Predicted MIP Share '+model,
                                                ascending=False).reset_index(drop=True)
            results_sorted['MIP Rank '+model] = results_sorted.index+1

            results = results.merge(results_sorted, on=['Player','Season','MIP Share','MIP Rank'])

        final_results = pd.concat([final_results,results], ignore_index=True)

        if i == n_seasons_to_test:
            break

        i = i + 1
    
    np.savetxt(path_data+sep+r'Models\params.csv', best_params, delimiter =', ', fmt ='% s')
    
    return final_results, metrics, best_params

In [68]:
def average_metrics(metrics):
    # Averages of each of the models
    final_metrics = pd.DataFrame()
    for model in metrics['Model'].unique():
        metric = metrics[metrics['Model']==model]
        rmse = round(metric['RMSE'].mean(),3)
        r2 = round(metric['R²'].mean(),3)

        dict_met = {'Model': [model],
                    'RMSE': [rmse],
                    'R²': [r2]}

        data_frame = pd.DataFrame(data=dict_met)
        final_metrics = pd.concat([final_metrics,data_frame], ignore_index=True)
    return final_metrics

In [69]:
pd.options.display.max_rows = 50

final_results, metrics, best_params = func_models(data, seasons, models, 1)
final_metrics = average_metrics(metrics)
final_metrics

68
FM:
SVR(C=0.5, degree=2)
SEP: 
/
MODEL: 
SVM
FM:
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)
SEP: 
/
MODEL: 
XGBoost
FM:
RandomForestRegressor(max_features=3, n_estimators=25, oob_score=True)
SEP: 
/
MODEL: 
Random Forest
FM:
AdaBoostRegressor(learning_rate=0.2)
SEP: 
/
MODEL: 
AdaBoost
FM:
Gradie

,Model,RMSE,R²
0,SVM,0.108,0.301
1,XGBoost,0.100,0.400
2,Random Forest,0.102,0.376
3,AdaBoost,0.112,0.242
4,Gradient Boosting,0.116,0.187


In [89]:
def create_rank(final_results, n_rank):
    rank_final = pd.DataFrame()

    for season in final_results['Season'].unique():
        temp = final_results[final_results['Season']==season]
        rank = pd.DataFrame()
        rank_real = temp.sort_values(by='MIP Share', ascending=False)[:n_rank].reset_index(drop=True)
        rank['MIP Rank Real'] = rank_real['Player']
        rank['MIP Share Real'] = rank_real['MIP Share']
        for model in models:
            try:
                temp2 = temp.sort_values(by='Predicted MIP Share '+model, ascending=False)[:n_rank].reset_index(drop=True)
                rank['MIP Rank '+model] = temp2['Player']
                rank['MIP Share '+model] = round(temp2['Predicted MIP Share '+model],3)
            except:
                continue

        rank['Season'] = season    
        rank_final = pd.concat([rank_final, rank], ignore_index=True)
    
    return rank_final

In [92]:
rank_final = create_rank(final_results, 3)
rank_final

,MIP Rank Real,MIP Share Real,MIP Rank SVM,MIP Share SVM,MIP Rank XGBoost,MIP Share XGBoost,MIP Rank Random Forest,MIP Share Random Forest,MIP Rank AdaBoost,MIP Share AdaBoost,MIP Rank Gradient Boosting,MIP Share Gradient Boosting,Season
0,Monta Ellis,0.546,Kevin Martin,0.398,Kevin Martin,0.326,Kevin Martin,0.404,Kevin Martin,0.330,Kevin Martin,0.410,2006-07
1,Kevin Martin,0.541,Al Jefferson,0.222,Deron Williams,0.205,Andre Iguodala,0.214,Zach Randolph,0.251,Deron Williams,0.316,2006-07
2,Deron Williams,0.157,Deron Williams,0.215,Andre Iguodala,0.194,Monta Ellis,0.173,Deron Williams,0.244,Zach Randolph,0.281,2006-07
3,Hedo Türkoğlu,0.608,Hedo Türkoğlu,0.337,Ronnie Brewer,0.183,Richard Jefferson,0.181,Mike Dunleavy,0.225,LaMarcus Aldridge,0.228,2007-08
4,Rudy Gay,0.267,Richard Jefferson,0.266,Rudy Gay,0.155,Rudy Gay,0.171,Rudy Gay,0.179,Rudy Gay,0.195,2007-08
5,LaMarcus Aldridge,0.205,Jason Richardson,0.231,LaMarcus Aldridge,0.103,LaMarcus Aldridge,0.142,Ronnie Brewer,0.127,Ronnie Brewer,0.160,2007-08
6,Danny Granger,0.602,Brandon Roy,0.250,Danny Granger,0.188,Devin Harris,0.084,Kevin Durant,0.126,Danny Granger,0.178,2008-09
7,Devin Harris,0.560,Troy Murphy,0.189,Kevin Durant,0.160,Danny Granger,0.066,John Salmons,0.118,Kevin Durant,0.144,2008-09
8,Kevin Durant,0.137,Danny Granger,0.186,Devin Harris,0.077,David Lee,0.062,Devin Harris,0.104,David Lee,0.135,2008-09
9,Aaron Brooks,0.655,Kevin Durant,0.274,Kevin Durant,0.217,Channing Frye,0.229,Kevin Durant,0.178,Brook Lopez,0.183,2009-10


##  Old Seasons

In [72]:
def temporadas_antigas(data, seasons, models, n_seasons_to_test):
    final_results = pd.DataFrame()
    metrics = pd.DataFrame()
    best_params = []
    i = 1

    for season in seasons:
            
        # Separating training and testing bases
        season_test = season

        data_train = data[data['Season']!=season_test]
        data_test = data[data['Season']==season_test]


        #TESTING FOR NOW
        ###
        ####
        regular_diffs = ['PTS_diff', 'AST_diff', 'TRB_diff']
        final_drops = []

        X_train = data_train.drop(['MIP Share','MIP Rank','Player','Season',]+final_drops, axis=1)#'G_prev','FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME'], axis=1)
        y_train = data_train['MIP Share']

        X_test = data_test.drop(['MIP Share','MIP Rank','Player','Season']+final_drops, axis=1)#,'FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME','G_prev'], axis=1)
        y_test = data_test['MIP Share']

        initial_results = data_test[['Player','Season','MIP Share','MIP Rank']]
        results = initial_results.copy()

        scaler = StandardScaler()

        scaled_X_train = scaler.fit_transform(X_train)
        scaled_X_test = scaler.transform(X_test)

        for model in models:
            
            if model=='Elastic Net':
                continue
            
            # Opening the models already created          
            current_model = pickle.load(open(path_data+sep+'Models'+sep+model+'.dat', 'rb'))
                
            current_model.fit(scaled_X_train, y_train)
            y_pred = current_model.predict(scaled_X_test)
            metrics = func_metrics(y_test, y_pred, metrics, model, season)

            apoio = initial_results.copy()
            apoio['Predicted MIP Share '+model] = pd.Series(y_pred).values

            results_sorted = apoio.sort_values(by='Predicted MIP Share '+model,
                                                ascending=False).reset_index(drop=True)
            results_sorted['MIP Rank '+model] = results_sorted.index+1

            results = results.merge(results_sorted, on=['Player','Season','MIP Share','MIP Rank'])

        final_results = pd.concat([final_results,results], ignore_index=True)

        if i == n_seasons_to_test:
            break

        i = i + 1
        
    return final_results, metrics

In [73]:
pd.options.display.max_rows = 100
pd.options.display.max_seq_items = 100
final_results, metrics = temporadas_antigas(data, seasons, models, 18)
final_metricas = average_metrics(metrics)
final_metricas

,Model,RMSE,R²
0,SVM,0.128,0.230
1,XGBoost,0.121,0.303
2,Random Forest,0.126,0.260
3,AdaBoost,0.126,0.246
4,Gradient Boosting,0.121,0.311


In [74]:
pd.options.display.max_rows = 400

rank_final = create_rank(final_results, 3)
rank_final

,MIP Rank Real,MIP Share Real,MIP Rank SVM,MIP Share SVM,MIP Rank XGBoost,MIP Share XGBoost,MIP Rank Random Forest,MIP Share Random Forest,MIP Rank AdaBoost,MIP Share AdaBoost,MIP Rank Gradient Boosting,MIP Share Gradient Boosting,Season
0,Monta Ellis,0.546,Kevin Martin,0.398,Kevin Martin,0.326,Kevin Martin,0.404,Kevin Martin,0.330,Kevin Martin,0.410,2006-07
1,Kevin Martin,0.541,Al Jefferson,0.222,Deron Williams,0.205,Andre Iguodala,0.214,Zach Randolph,0.251,Deron Williams,0.316,2006-07
2,Deron Williams,0.157,Deron Williams,0.215,Andre Iguodala,0.194,Monta Ellis,0.173,Deron Williams,0.244,Zach Randolph,0.281,2006-07
3,Hedo Türkoğlu,0.608,Hedo Türkoğlu,0.337,Ronnie Brewer,0.183,Richard Jefferson,0.181,Mike Dunleavy,0.225,LaMarcus Aldridge,0.228,2007-08
4,Rudy Gay,0.267,Richard Jefferson,0.266,Rudy Gay,0.155,Rudy Gay,0.171,Rudy Gay,0.179,Rudy Gay,0.195,2007-08
5,LaMarcus Aldridge,0.205,Jason Richardson,0.231,LaMarcus Aldridge,0.103,LaMarcus Aldridge,0.142,Ronnie Brewer,0.127,Ronnie Brewer,0.160,2007-08
6,Danny Granger,0.602,Brandon Roy,0.250,Danny Granger,0.188,Devin Harris,0.084,Kevin Durant,0.126,Danny Granger,0.178,2008-09
7,Devin Harris,0.560,Troy Murphy,0.189,Kevin Durant,0.160,Danny Granger,0.066,John Salmons,0.118,Kevin Durant,0.144,2008-09
8,Kevin Durant,0.137,Danny Granger,0.186,Devin Harris,0.077,David Lee,0.062,Devin Harris,0.104,David Lee,0.135,2008-09
9,Aaron Brooks,0.655,Kevin Durant,0.274,Kevin Durant,0.217,Channing Frye,0.229,Kevin Durant,0.178,Brook Lopez,0.183,2009-10


In [75]:
data_test = data[data['Season']=='2023-24']
data_train = data[data['Season']!='2023-24']

regular_diffs = ['PTS_diff', 'AST_diff', 'TRB_diff']
final_drops = []

X_test = data_test.drop(['MIP Share','MIP Rank','Player','Season',] + final_drops, axis=1) #'G_prev' 'FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME'], axis=1)
X_train = data_train.drop(['MIP Share','MIP Rank','Player','Season' ,] + final_drops, axis=1) #'G_prev'  'FGA_PERGAME', 'PER_ADVANCED', 'FG%_PERGAME'], axis=1)

scaler = StandardScaler()
scaler.fit(X_train)
feature_names = X_test.columns
print(feature_names)

scaled_X_test = scaler.transform(X_test)
scaled_X_test_df = pd.DataFrame(scaled_X_test, columns=feature_names)
#scaled_X_test_df

Index(['Age', 'G', 'MP_PERGAME', 'FG_PERGAME', 'FGA_PERGAME', 'FG%_PERGAME',
       '3P_PERGAME', '3PA_PERGAME', '3P%_PERGAME', '2P_PERGAME', '2PA_PERGAME',
       '2P%_PERGAME', 'FT_PERGAME', 'FT%_PERGAME', 'ORB_PERGAME',
       'DRB_PERGAME', 'TRB_PERGAME', 'AST_PERGAME', 'BLK_PERGAME',
       'TOV_PERGAME', 'PTS_PERGAME', 'G_prev', 'Seasons Played',
       'PER_ADVANCED', 'AST%_ADVANCED', 'USG%_ADVANCED', 'OWS_ADVANCED',
       'WS_ADVANCED', 'WS/48_ADVANCED', 'BPM_ADVANCED', 'VORP_ADVANCED',
       'G_diff', 'GS_diff', 'MP_diff', 'FG_diff', 'FGA_diff', '3P_diff',
       '3PA_diff', '2P_diff', '2PA_diff', 'FT_diff', 'FTA_diff', 'DRB_diff',
       'TRB_diff', 'AST_diff', 'BLK_diff', 'TOV_diff', 'PTS_diff',
       'PTS_diff_BS', 'PER_diff', 'AST%_diff', 'USG%_diff', 'OWS_diff',
       'STL%_diff', 'BLK%_diff', '3PAr_diff', 'ORB%_diff', 'OBPM_diff',
       'DBPM_diff', 'DWS_diff', 'WS_diff', 'WS/48_diff', 'BPM_diff',
       'VORP_diff', 'Seed', 'Previous Seed', 'Seed Difference', 'PCT'

In [81]:
svm_model = pickle.load(open(path_data+sep+'Models'+sep+'Gradient Boosting.dat','rb'))


In [82]:
# Define a prediction function
def predict_fn(X):
    return svm_model.predict(X)

# Use SHAP's KernelExplainer
explainer = shap.KernelExplainer(predict_fn, shap.sample(scaled_X_test_df, 100))
shap_values = explainer.shap_values(scaled_X_test_df)

#explainer = shap.Explainer(svm_model.predict, scaled_X_test)
#shap_values = explainer(scaled_X_test)

  0%|          | 0/24 [00:00<?, ?it/s]

In [83]:
scaled_X_test_df

,Age,G,MP_PERGAME,FG_PERGAME,FGA_PERGAME,FG%_PERGAME,3P_PERGAME,3PA_PERGAME,3P%_PERGAME,2P_PERGAME,2PA_PERGAME,2P%_PERGAME,FT_PERGAME,FT%_PERGAME,ORB_PERGAME,DRB_PERGAME,TRB_PERGAME,AST_PERGAME,BLK_PERGAME,TOV_PERGAME,PTS_PERGAME,G_prev,Seasons Played,PER_ADVANCED,AST%_ADVANCED,USG%_ADVANCED,OWS_ADVANCED,WS_ADVANCED,WS/48_ADVANCED,BPM_ADVANCED,VORP_ADVANCED,G_diff,GS_diff,MP_diff,FG_diff,FGA_diff,3P_diff,3PA_diff,2P_diff,2PA_diff,FT_diff,FTA_diff,DRB_diff,TRB_diff,AST_diff,BLK_diff,TOV_diff,PTS_diff,PTS_diff_BS,PER_diff,AST%_diff,USG%_diff,OWS_diff,STL%_diff,BLK%_diff,3PAr_diff,ORB%_diff,OBPM_diff,DBPM_diff,DWS_diff,WS_diff,WS/48_diff,BPM_diff,VORP_diff,Seed,Previous Seed,Seed Difference,PCT
0,0.620726,0.204538,0.947097,2.255153,2.269745,-0.325749,1.679785,1.775825,0.390231,1.091229,1.025849,0.181135,0.650906,-0.522770,-0.553523,-0.462374,-0.522298,0.955824,-0.443734,0.695415,2.146960,0.287558,1.258577,0.519592,0.739823,1.779115,-0.307432,0.049549,-0.265598,0.441483,0.655207,-0.096253,-0.586499,-0.447563,-1.320221,-0.106663,2.153215,1.859019,-2.506857,-1.338984,-1.805972,-1.480478,-0.713934,-0.485396,-1.269125,0.051516,-0.632282,-1.158839,-0.598445,-1.848502,-1.616093,-0.554963,-2.148464,2.790629,0.529666,1.277141,0.831428,-1.672080,1.319217,1.092137,-1.221558,-1.262491,-0.674933,-0.427092,0.350033,-0.926186,-1.597553,0.518390
1,-0.572857,-0.247810,1.376474,1.849130,2.088860,-0.637362,1.781694,1.895999,0.417738,0.636945,0.694318,-0.269775,0.989725,1.022887,-0.936905,-0.720039,-0.841725,1.228047,-0.273558,-0.569304,1.986632,-0.926596,-0.207193,0.425363,0.885068,1.070748,0.918020,0.635470,0.356285,0.620585,0.790601,0.640664,0.522296,-0.128318,0.386377,1.223132,-0.070619,1.019480,0.305038,0.560196,1.143774,0.888420,-0.189196,-0.149658,2.071284,1.124890,0.022382,0.663811,0.802687,0.289467,1.967404,0.739270,0.474706,0.297942,1.128753,-0.171043,0.216611,0.661162,0.335263,0.152755,0.433052,-0.021970,0.767853,1.220351,0.017698,-1.367932,-1.734186,0.584372
2,-1.368579,0.883060,0.705572,1.104753,1.274877,-0.533491,0.253063,0.413853,0.183925,0.896536,1.059002,-0.441550,1.260780,-0.677336,-0.457677,0.671352,0.294016,0.865083,-0.103381,1.398037,1.230803,0.194162,-1.184373,-0.359880,0.739823,1.472156,-1.250087,-0.389892,-0.825293,-0.185372,0.045933,0.476905,-0.309300,-0.744004,-0.223122,-0.494521,-0.070619,-0.379752,-0.183987,-0.297498,-1.671893,-1.593282,-0.608986,-0.821133,1.027406,0.051516,-0.195840,-0.703176,-0.198122,0.099425,1.513158,0.005871,0.117001,0.297942,0.230123,-0.199720,-0.244502,0.505613,0.950234,1.092137,0.524975,0.598290,0.956042,1.117386,-0.093080,1.061672,1.408369,0.584372
3,1.018587,0.656886,0.893425,1.510777,1.636647,-0.450394,1.374059,1.495419,0.348970,0.572047,0.561706,0.095247,-0.365550,0.143051,-0.649368,-0.050110,-0.273855,1.318788,-0.613911,0.695415,1.207899,0.380955,1.258577,-0.234241,1.001264,0.740176,-0.307432,-0.536372,-0.949670,-0.006270,0.249024,0.149386,-0.467700,-1.177264,-1.442121,-1.048602,1.041298,1.019480,-2.017832,-1.829095,-0.331099,-0.239626,-0.818882,-0.821133,-0.434023,-0.306275,0.022382,-0.976574,-0.598445,-0.708252,0.478487,0.005871,-0.300322,-0.200595,0.379894,1.105079,0.293463,-0.116585,-0.648692,-1.099754,-0.715982,-0.513010,-0.361284,-0.427092,0.682369,0.178180,-0.641123,-0.141425
4,-0.970718,-0.700158,-0.260527,1.104753,1.395467,-0.803556,0.966424,1.054781,0.355847,0.442252,0.661165,-0.699213,0.718670,0.880211,-1.032751,-0.926171,-1.019184,-0.269181,-0.784087,-0.288255,1.207899,-1.206786,-0.695783,-0.736796,-0.180062,1.661054,-1.250087,-1.415254,-1.488635,-1.304755,-1.240313,0.558784,1.235093,2.357228,3.677673,3.938132,2.153215,2.698558,2.505652,2.581904,1.143774,1.114029,0.545438,0.437883,0.818631,0.051516,0.895267,3.534485,2.904384,-0.375679,0.251363,0.221577,-0.240704,0.297942,0.230123,0.459849,0.447167,0.038964,0.212268,0.152755,-0.118484,-0.280412,0.140554,-0.324127,0.793147,-0.705313,-1.870819,-0.405351
5,-1.368579,-1.039419,0.034670,1.375435,0.671927,1.169993,-0.766024,-0.627654,-0.

In [84]:
shap.summary_plot(shap_values, scaled_X_test_df,plot_type='violin') #X_test, plot_type='violin') # Top 20 variables

In [85]:
shap.summary_plot(shap_values, X_test, plot_type='violin', max_display=len(X_test.columns)) # All features